In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import pylab as pl
import scipy.cluster.hierarchy as hier
import scipy.spatial.distance as dist
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
gene = pd.read_excel('gene_expression_table4.xlsx') # this is an edited version of the raw data table
del gene['Unnamed: 9'] # deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:14]
print(just_expression)
just_expression.as_matrix  # transformed dataframe with expression values to array
#just_expression.shape
#dist_matrix= dist.pdist(just_expression)
#dist_square_matrix = dist.squareform(dist_matrix)

# PCA and classification

In [ ]:
#From Jose
gene = pd.read_excel('gene_expression_table4.xlsx')
del gene['Unnamed: 9']
# deleted column without any content
#append each column except the first column which is the gene ID
just_expression=gene.iloc[0:373,3:15]
just_expression


In [ ]:
#flipping rows and columns so the columns are the genes

JEflip = just_expression.transpose()
print(JEflip)

JEflipar = JEflip.values
JEflipar

In [ ]:
#Making a new array to label gene expression under low or high BRCA1 (rows in JEflip)
brca = np.array(['BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-',
         'BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+'])

In [ ]:
from sklearn.decomposition import PCA
JEflip_pca = PCA()
JEflippc = JEflip_pca.fit_transform(JEflipar) # 12x12 matrix
print(JEflip_pca.explained_variance_ratio_)
print(np.cumsum(JEflip_pca.explained_variance_ratio_))

In [ ]:
fg = plt.figure()

plt.plot(np.cumsum(JEflip_pca.explained_variance_ratio_))
plt.xlabel("PC's")
plt.ylabel('fraction var explained')
plt.title("PC's of 373 genes")

JEvar = JEflip_pca.explained_variance_ratio_
cumvar = np.cumsum(JEvar) 
for i in range (12):
    if cumvar[i] >= 0.9:
        print (i+1, 'components are need to explain 90% of the variance')
        break
        
# 5 of the 12 PCs needed to explain 90% of the variance
# but the first PC already explains more than 75% of the variance

In [ ]:
# Plot function from homework 5 key

fg = plt.figure()

fg.add_subplot(3,1,1)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,1], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC2')

fg.add_subplot(3,1,2)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,5], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC6')

fg.add_subplot(3,1,3)
for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,0], 
             JEflippc[brca==c,11], 'o')
plt.xlabel('PC1')    
plt.ylabel('PC12')

plt.tight_layout()

#When the indiviual PCs are compoared with the first PC, always a clear separation into 
#2 groups of 6 samples - corresponds to BRCA1+ or BRCA1-.
#Here example of 3

In [ ]:
for i in np.unique(brca):
    plt.plot(JEflippc[brca==i,0], 
             JEflippc[brca==i,1], 'o' )
plt.xlabel('PC1')    
plt.ylabel('PC2')
plt.title('PC1 and PC2, coloured by BRCA1 status')


In [ ]:
#Function from homework 5 key

for c in np.unique(brca):
    plt.plot(JEflippc[brca==c,1], 
             JEflippc[brca==c,2], 'o')
plt.xlabel('PC2')    
plt.ylabel('PC3')

# When compared between other PCs that are not the first PC, less clear separation

In [ ]:
# test and train data specifically for this dataset

#train_ind1 = np.random.choice(np.arange(0,6), size=5, replace=False, p=None)
#train_ind2 = np.random.choice(np.arange(6,12), size=5, replace=False, p=None)

#train_ind = np.concatenate((train_ind1, train_ind2), axis =0)
#test_ind=  np.setdiff1d(np.arange(0,12), train_ind)

#print(train_ind)
#print(test_ind)

In [ ]:
# Function from homework 5 key 

from sklearn.decomposition import PCA

#def cross_val_class_accuracy(model, X, y, r, test_frac, reps):

def cross_val_class_accuracy(model, X, y, r, reps):    
    JEflip_pca = PCA()
    
    score = np.array([])
    for i in range(reps):
        train_ind1 = np.random.choice(np.arange(0,6), size=5, replace=False, p=None)
        train_ind2 = np.random.choice(np.arange(6,12), size=5, replace=False, p=None)
        train_ind = np.concatenate((train_ind1, train_ind2), axis =0)
        test_ind=  np.setdiff1d(np.arange(0,12), train_ind)
    
        y_train = y[train_ind]
        y_test = y[test_ind]
        
        X_train = JEflip_pca.fit_transform(X[train_ind, :])[:, :r]
        X_test = JEflip_pca.transform(X[test_ind, :])[:, :r]
        
        model.fit(X_train, y_train)
        
        pred = model.predict(X_test)
        this_score = sum(pred == y_test) / len(y_test)
       
        score = np.append(score, this_score)
    return score



In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [ ]:
score = cross_val_class_accuracy(KNeighborsClassifier(n_neighbors=2), JEflipar, 
                                brca, 10, 200)
print(score.mean())



In [ ]:
reps = 200

lda_r5_scores = cross_val_class_accuracy(LinearDiscriminantAnalysis(), 
                                          JEflipar, brca, 5,
                                          reps)
lda_r10_scores = cross_val_class_accuracy(LinearDiscriminantAnalysis(), 
                                          JEflipar, brca, 10,
                                            reps)
knn2_scores = cross_val_class_accuracy(KNeighborsClassifier(n_neighbors=2),
                                       JEflipar, brca, 10,
                                       reps)
knn10_scores = cross_val_class_accuracy(KNeighborsClassifier(n_neighbors=10),
                                       JEflipar, brca, 10,
                                        reps)
svm_scores = cross_val_class_accuracy(svm.SVC(kernel='linear'),
                                      JEflipar, brca, 10,
                                      reps)
tree_scores = cross_val_class_accuracy(DecisionTreeClassifier(max_depth=3),
                                       JEflipar, brca, 10,
                                       reps)

In [ ]:
classifiers = ('lda_r5', 'lda_r10', 'knn2', 'knn10', 'svm', 'tree')
scores = [lda_r5_scores.mean(), lda_r10_scores.mean(), 
          knn2_scores.mean(), knn10_scores.mean(),
         svm_scores.mean(), tree_scores.mean()]
scores_err = [lda_r5_scores.std(), lda_r10_scores.std(), 
          knn2_scores.std(), knn10_scores.std(),
         svm_scores.std(), tree_scores.std()]


plt.bar(range(len(classifiers)), scores, 
        yerr=scores_err,
        align='center', alpha=0.4,)
plt.xticks(range(len(classifiers)), classifiers)
plt.ylabel('Cross-validated accuracy')
plt.xlabel('Classification Algorithm')
plt.title('Cross validation of four different kinds of classification models')

In [ ]:
# lda with 5 pcs does well
# knn with 2 neighbours and 10 pcs does well
# svm and decision tree do well 

# Difference of Means

In [ ]:
BRCA1_plus= just_expression.iloc[:,0:6]

BRCA1_minus= just_expression.iloc[:,7:12]

BRCA1_plus_means=np.mean(BRCA1_plus.T)

BRCA1_minus_means=np.mean(BRCA1_minus.T)

concatenated_means= np.c_[BRCA1_plus_means,BRCA1_minus_means]

expression_change_abs= np.abs(np.diff(concatenated_means))

expression_change_raw= np.diff(concatenated_means)

concatenated_means=np.append(expression_change_abs,expression_change_raw,1) # appended raw and abs value of means

In [ ]:
labels=gene.iloc[0:373,0:3] # isolated gene names and functions
names=['mean','raw mean']
mean_pandas2= pd.DataFrame(concatenated_means,columns=names) # transformed concatenated means to dataframe

mean_append= np.append(labels,mean_pandas2,1) # appended gene labels to concatenated means

names2= ['gene name', 'symbol','function','abs value mean difference','raw mean difference']
labels_and_means =pd.DataFrame(mean_append,columns=names2)

sorted_means=labels_and_means.sort(['abs value mean difference'],ascending=False)
#print(sorted_means)
selected_top_20 = sorted_means.iloc[0:20,]
selected_top_20

In [ ]:
#from pandas import *
#idx = Int64Index([np.arange(373)])9
#index_sorted_means = DataFrame(index = np.arange(sorted_means.size), data =(sorted_means))
#index_sorted_means
#sorted_means
names_labels=['labels']
index_sorted_means = pd.DataFrame(np.arange(373),columns=names_labels)

data_names= ['gene order (high difference to low)','gene name', 'symbol','function','abs value mean difference','raw mean difference']
a = np.append(index_sorted_means, sorted_means,1)
my_labels= pd.DataFrame(a, columns=data_names)
my_labels2= my_labels[my_labels.function !='unknown']
my_labels2_select = my_labels2.loc[0:20,]
my_labels2_select

In [ ]:
from ggplot import *
limit_graph = my_labels2.iloc[0:373, :]
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'abs value mean difference', color = 'function')) + \
    geom_point(size = 45) + \
    ggtitle("Difference of Means For All 373 Genes")

In [ ]:
from ggplot import *
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'raw mean difference', color = 'function')) + \
    geom_point(size = 100) + \
    ggtitle("Difference of Mean Gene Expression For All Genes (BRCA high - BRCA low)", ) 

In [ ]:
from ggplot import *
limit_graph = my_labels2.iloc[0:20, :]
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'abs value mean difference', color = 'function')) + \
    geom_point(size = 45) + \
    ggtitle("Difference of Means For Top 20 Genes")

In [ ]:
from ggplot import *
ggplot(limit_graph, aes(x = 'gene order (high difference to low)', y = 'raw mean difference', color = 'function')) + \
    geom_point(size = 100) + \
    ggtitle("Difference of Mean Gene Expression For Top 20 (BRCA high - BRCA low)", ) 

# Lasso 

In [ ]:
#First make sure of the data to use
#array of 'brca1' did not work, so used numbers
#tried for numbers 0,1 or 99,100
#no difference, so should be ok to use 0,1

In [ ]:
# testing of brca data

brca2 = (99,99,99,99,99,99,100,100,100,100,100,100)

brca3 = (0,0,0,0,0,0,1,1,1,1,1,1)

from sklearn import linear_model
from sklearn.linear_model import Lasso

#lasso code from example on scikit learn directory website
#not actual usable code yet, used arbitrary alpha just to test

reg = linear_model.Lasso(alpha = 0.1)
reg.fit(JEflip, brca2)
#print(reg.coef_)
output = reg.coef_

reg2 = linear_model.Lasso(alpha = 0.1)
reg2.fit(JEflip, brca3)
#print(reg2.coef_)
output2 = reg2.coef_

output == output2

In [ ]:
# Making test and train data for the Lasso, on our specific dataset
# Choosing 5 train data from brca1-, and 5 train data from brca1+
# Then left with 1 brca1- cell line and 1 brca1+ cell line for test data

train_ind1 = np.random.choice(np.arange(0,6), size=5, replace=False, p=None)
train_ind2 = np.random.choice(np.arange(6,12), size=5, replace=False, p=None)
print(train_ind1)
print(train_ind2)

train_ind = np.concatenate((train_ind1, train_ind2), axis =0)
test_ind=  np.setdiff1d(np.arange(0,12), train_ind)

In [ ]:
expression_train=JEflipar[train_ind,:] #test and train indices used on our data
expression_test=JEflipar[test_ind,:]

In [ ]:
brca_end = np.array([0,0,0,0,0,0,1,1,1,1,1,1])

#brca_end = np.array(['BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-','BRCA1-',
         #'BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+','BRCA1+'])

brcatrain = brca_end[train_ind]
brcatest = brca_end[test_ind]

#brcatrain will always be array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
#brcatest will always be array([0, 1])


In [ ]:
from sklearn.preprocessing import scale
from sklearn import cross_validation
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

lasso = Lasso(max_iter=10000, normalize=True)

#importing all of these according to document by R.J. Crouser

In [ ]:
# code from a document created by R. Jordan Crouser at Smith College for SDS293: Machine Learning 
# found here: http://www.science.smith.edu/~jcrouser/SDS293/labs/lab10/Lab%2010%20-%20Ridge%20Regression%20and%20the%20Lasso%20in%20Python.pdf

#looking at random alphas
alphas = 10**np.linspace(10,-2,100)*0.5
lasso = Lasso(max_iter=10000, normalize=True)
coefs = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(expression_train), brcatrain)
    coefs.append(lasso.coef_)
    

ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
plt.title ('100 alphas')

#np.shape(coefs)
#v=coefs
#print(v)



In [ ]:
# lassocv - 10 fold cross validation to find right alpha

lassocv = LassoCV(alphas=None, cv=10, max_iter=100000, normalize=True)

lassocv.fit(expression_train, brcatrain)
lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(expression_train, brcatrain)
mean_squared_error(brcatest, lasso.predict(expression_test))

In [ ]:
lassocv.alpha_

# Rest of lasso code
